Training a Deep Learning Model which can then be used as a testing case for visualization. This model consists of 4 Dense layers with 8, 4, 2 and 1 unit respectively. It is a binary classification model trained on the Titanic Dataset (available in Seaborn).

In [1]:
import tensorflow as tf
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
# Data loading and feature exploration
data = sns.load_dataset('titanic')
data.iloc[5:15, :]

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
5,0,3,male,NaN,0,0,8.4583,Q,Third,man,True,NaN,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,First,man,True,E,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,Third,child,False,NaN,Southampton,no,False
8,1,3,female,27.0,0,2,11.1333,S,Third,woman,False,NaN,Southampton,yes,False
9,1,2,female,14.0,1,0,30.0708,C,Second,child,False,NaN,Cherbourg,yes,False
10,1,3,female,4.0,1,1,16.7000,S,Third,child,False,G,Southampton,yes,False
11,1,1,female,58.0,0,0,26.5500,S,First,woman,False,C,Southampton,yes,True
12,0,3,male,20.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True
13,0,3,male,39.0,1,5,31.2750,S,Third,man,True,NaN,Southampton,no,False
14,0,3,female,14.0,0,0,7.8542,S,Third,child,False,NaN,Southampton,no,True


In [3]:
# Data Cleaning

# Dropping irrelevant columns and rows containing NaN values
data = data.drop(["class", "who", "adult_male", "deck", "embark_town", "alive"], axis = 1)
data = data.dropna()

# Converting categorical features into numerical ones
data["sex"] = data["sex"].map(
    lambda x:
        0 if x == "male" else 1
)
data["alone"] = data["alone"].map(
    lambda x: 0 if x == False else 1
)
unique_embarked = data["embarked"].unique()
embkd = {unique_embarked[i]:i for i in range(len(unique_embarked))}
data["embarked"] = data["embarked"].map(
    lambda x: embkd[x]
)
data

,survived,pclass,sex,age,sibsp,parch,fare,embarked,alone
0,0,3,0,22.0,1,0,7.2500,0,0
1,1,1,1,38.0,1,0,71.2833,1,0
2,1,3,1,26.0,0,0,7.9250,0,1
3,1,1,1,35.0,1,0,53.1000,0,0
4,0,3,0,35.0,0,0,8.0500,0,1
...,...,...,...,...,...,...,...,...,...
885,0,3,1,39.0,0,5,29.1250,2,0
886,0,2,0,27.0,0,0,13.0000,0,1
887,1,1,1,19.0,0,0,30.0000,0,1
889,1,1,0,26.0,0,0,30.0000,1,1


In [4]:
# Feature-Label extraction and Train-Test data splitting
y = data["survived"]
x = data.drop(["survived"], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=1)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((534, 8), (178, 8), (534,), (178,))

In [5]:
# Building the model
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(units=8))
model.add(tf.keras.layers.Dense(units=4))
model.add(tf.keras.layers.Dense(units=2, use_bias=False, activation="sigmoid"))
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))
model.build(input_shape=(None, 8))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 72        
                                                                 
 dense_1 (Dense)             (None, 4)                 36        
                                                                 
 dense_2 (Dense)             (None, 2)                 8         
                                                                 
 dense_3 (Dense)             (None, 1)                 3         
                                                                 
Total params: 119
Trainable params: 119
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# Saving the untrained form of our model. All the weights and biases are randomly initialized.
model.save('./../models/titanic/untrained')

In [7]:
model.compile(
        optimizer='adam',
        metrics=['accuracy'],
        loss='binary_crossentropy'
)

In [ ]:
model.fit(x_train, y_train, epochs=200)

In [9]:
eval = model.evaluate(x_test, y_test)
print("Loss and accuracy on testing data:", eval)

6/6 [==============================] - 0s 2ms/step - loss: 0.4752 - accuracy: 0.8034
Loss and accuracy on testing data: [0.47524768114089966, 0.8033707737922668]


In [ ]:
# Saving the trained form of our model. All the weights and biases are optimized.
model.save('./../models/titanic/trained')